# IMPORTS

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.cluster import KMeans



## LEITURA DOS DADOS

In [37]:
df = pd.read_csv('../../RawData/train_radiomics_occipital_CONTROL.csv') 
x_train = df.drop(columns=['Transition'])

# Tratamento

### Single Value Colums 

In [38]:
df = df.loc[:, df.nunique() > 1]


#  Analise das colunas que tem menos de 50 valores unicos 
n = df.nunique()
#for col, e in n.items():
#    if e < 50:  
#        print(f"Coluna: {col}, Valores Unicos : {e}")

### AGE BINING  ( secalhar isto era no 2 )

In [39]:

age_bins = [0, 65, 75, 85, 100]
# BINS_SIZER = ['<65', '65-74', '75-84', '85+']
age_labels = [60, 70, 80, 90] # VALOR MEDIO DO BIN 
df['Age'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels).astype(int)

#print(df['Age'])

### Colunas Categoricas 

In [40]:
# Analisar a contagem de valores únicos para cada coluna categórica

# Identificar as colunas categóricas
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

#for col in categorical_columns:
#    print(f"\nColuna: {col}")

#colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash'] 

colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'] 

# ** Bounding Box
#
# ** as colunas do 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'
# ** Deveriam ser retiradas, mas o bounding box pode ser importante para a zona de maior ativação do Alzimeir 
# **  ja a de centro de maxima devem ser muito correlacionados, por isso devem ser retirados mais para a frente 
# *TODO acabei por retirar para correr melhor os modelos, mas analisar se se deve retirar ou nao 

df.drop(columns=colunas_catagoricas_a_remover,axis= 1 , inplace= True)


### Eliminar Transition CN-MCI 

Como este tipo de precisao não vai ser realiza, decidimos retirar todas as linhas com ela relacinadas para nao confundir o nosso modelo

### Label Encoding

In [42]:
label_mapping = {
    'CN-CN': 0,
    'AD-AD': 1,
    'MCI-AD': 2,
    'MCI-MCI': 3,
    'CN-MCI' : 4
}

df['Transition'] = df['Transition'].map(label_mapping)



# Models

In [43]:

#experiment = setup(df, target='Transition')

In [44]:
#best = compare_models()

Com o tratamento atual, o pycaret diz que os melhores modelos são :
- Random Forest Classifier
- Extra Trees Classifier
- Gradient Boosting Classifier 
- K Neighbors Classifier
- Extreme Gradient Boosting 


### Aplicação dos modelos (default) ao dataset tratado

In [45]:


X = df.drop('Transition', axis=1)
y = df['Transition']


# Dividir o conjunto de dados em treino e teste com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=34, stratify=y)


# Inicializar dicionário para armazenar as pontuações de cross-validation do modelo
model_cross_score = {}


### Funcao Matrix de Confusao 

In [46]:
def plot_confusion_matrix_with_labels(confusion_matrix, label_mapping):
    # Criar um mapeamento inverso
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    
    # Aplicar o mapeamento inverso na matriz de confusão
    cm_with_labels = np.zeros_like(confusion_matrix, dtype=object)
    for i in range(confusion_matrix.shape[0]):
        for j in range(confusion_matrix.shape[1]):
            cm_with_labels[i, j] = f"{reverse_label_mapping[i]} (Pred: {reverse_label_mapping[j]})"

    # Criar um DataFrame para facilitar a visualização
    df_cm = pd.DataFrame(confusion_matrix, index=reverse_label_mapping.values(), columns=reverse_label_mapping.values())
    
    # Plotar a matriz de confusão
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix with Labels')
    plt.show()

#### Random Forest Classifier

In [47]:

rf_model = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=34)

rf_score = cross_val_score(rf_model, X_train, y_train, cv=10,scoring='f1_macro')
print(rf_score)

model_cross_score['random_forest'] = rf_score





[0.1952381  0.35735646 0.22007576 0.40432331 0.30300752 0.23011364
 0.49047619 0.24242424 0.47321429 0.42638889]


#### Extra Trees Classifier

In [48]:
et_model = ExtraTreesClassifier(criterion='gini', max_depth=20, random_state=34)

et_score = cross_val_score(et_model, X_train, y_train, cv=10,scoring='f1_macro')

print(et_score)

model_cross_score['Extra Trees'] = et_score

[0.21785714 0.2748538  0.21637427 0.36428571 0.20021645 0.28039216
 0.49924242 0.19583049 0.29824561 0.18055556]


#### Gradient Boosting Classifier 

In [49]:
gb_model = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, random_state=34)

score_cross = cross_val_score(gb_model,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['Gradient_boosting'] = score_cross



[0.33730159 0.30140693 0.14583333 0.298779   0.18808275 0.21313131
 0.34166667 0.12692308 0.35946276 0.34027778]


#### K Neighbors Classifier

In [50]:
kn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')
k_night_croos_score = cross_val_score(kn_model, X_train, y_train, cv=10,scoring='f1_macro')

print("Acuracy k night", k_night_croos_score)


model_cross_score['k_night'] = k_night_croos_score




Acuracy k night [0.23611111 0.39761905 0.11025641 0.16666667 0.13392857 0.18154762
 0.38636364 0.05263158 0.15       0.33396465]


#### Extreme Gradient Boosting

In [51]:
xgb_model = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05,colsample_bytree=0.4, subsample=0.8, random_state=34)


score_cross = cross_val_score(xgb_model,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['XGBoost'] = score_cross



[0.3375     0.45555556 0.14930556 0.36940837 0.32046784 0.20098039
 0.41875    0.20779221 0.40198413 0.13392857]


SVC

In [ ]:


svc_moldel = SVC(kernel='rbf', random_state=34)


score_cross = cross_val_score(svc_moldel,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['SVC'] = score_cross



[0.11666667 0.12068966 0.12068966 0.13333333 0.13333333 0.125
 0.125      0.12068966 0.12068966 0.20833333]


In [55]:
def print_model_scores(model_scores):
    for model, scores in model_scores.items():
        max_score = np.max(scores)
        min_score = np.min(scores)
        diff = max_score - min_score
        avg_score = np.mean(scores)
        
        print(f"{model} (Max: {max_score:.4f}, Min: {min_score:.4f}, Diff: {diff:.4f}):")
        print(f"  Scores: {', '.join([f'{score:.4f}' for score in scores])}")
        print(f"  Average: {avg_score:.4f}")
        print('-' * 50)

# Chamar a função para imprimir o dicionário formatado
print_model_scores(model_cross_score)

random_forest (Max: 0.4905, Min: 0.1952, Diff: 0.2952):
  Scores: 0.1952, 0.3574, 0.2201, 0.4043, 0.3030, 0.2301, 0.4905, 0.2424, 0.4732, 0.4264
  Average: 0.3343
--------------------------------------------------
Extra Trees (Max: 0.4992, Min: 0.1806, Diff: 0.3187):
  Scores: 0.2179, 0.2749, 0.2164, 0.3643, 0.2002, 0.2804, 0.4992, 0.1958, 0.2982, 0.1806
  Average: 0.2728
--------------------------------------------------
Gradient_boosting (Max: 0.3595, Min: 0.1269, Diff: 0.2325):
  Scores: 0.3373, 0.3014, 0.1458, 0.2988, 0.1881, 0.2131, 0.3417, 0.1269, 0.3595, 0.3403
  Average: 0.2653
--------------------------------------------------
k_night (Max: 0.3976, Min: 0.0526, Diff: 0.3450):
  Scores: 0.2361, 0.3976, 0.1103, 0.1667, 0.1339, 0.1815, 0.3864, 0.0526, 0.1500, 0.3340
  Average: 0.2149
--------------------------------------------------
XGBoost (Max: 0.4556, Min: 0.1339, Diff: 0.3216):
  Scores: 0.3375, 0.4556, 0.1493, 0.3694, 0.3205, 0.2010, 0.4187, 0.2078, 0.4020, 0.1339
  Average